# Model training and registration
This notebook show the process for training the model, converting the model to ONNX and uploading the ONNX model to Azure Storage.

## Explore the training data
The following cells load the source CSV file into a Spark DataFrame and create a temporary view that can be used to query the data with Spark SQL.


In [3]:
df = spark.read.load('abfss://dev@<primary_storage>.dfs.core.windows.net/bronze/wwi-factsale.csv', format="csv"
## If header exists uncomment line bellow
, header=True, sep="|"
)

In [4]:
df.createOrReplaceTempView("facts")

In [5]:
display(spark.sql("SELECT * FROM facts WHERE `Customer Key` == '11' ORDER BY `Stock Item Key`"))

## Predict Quantity given Customer Key and Stock Item Key
In the following cells we load a subset of the data that just contains the fields needed for training. 



In [6]:
from pyspark.sql.functions import col
df3 = spark.sql("SELECT double(`Customer Key`) as customerkey, double(`Stock Item Key`) as stockitemkey, double(`Quantity`) as quantity FROM facts").where(col("quantity").isNotNull())
df3.cache()

DataFrame[customerkey: double, stockitemkey: double, quantity: double]

Next, we package the data into the format expected by SPark ML's LinearRegression. It requires a DataFrame with two columns- `features` and a column with the labels to predict (`quantity` in this case).


In [8]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['customerkey', 'stockitemkey'], outputCol = 'features')
df4 = vectorAssembler.transform(df3)
df5 = df4.select(['features', 'quantity'])
df5.show(10)

+-------------+--------+
|     features|quantity|
+-------------+--------+
|  [0.0,156.0]|     4.0|
|   [0.0,52.0]|    90.0|
|   [0.0,54.0]|    10.0|
|  [0.0,141.0]|    72.0|
|  [0.0,185.0]|     7.0|
|  [0.0,148.0]|     6.0|
| [141.0,51.0]|    80.0|
|[141.0,216.0]|     5.0|
| [141.0,94.0]|    10.0|
|[141.0,206.0]|     5.0|
+-------------+--------+
only showing top 10 rows

Now, we split our DataFrame into training and testing DataFrames.


In [9]:
trainingFraction = 0.7
testingFraction = (1-trainingFraction)
seed = 42

# Split the dataframe into test and training dataframes
df_train, df_test = df5.randomSplit([trainingFraction, testingFraction], seed=seed)

In the following cell, we train our LinearRegression model.


In [10]:
from pyspark.ml.regression import LinearRegression

lin_reg = LinearRegression(featuresCol = 'features', labelCol='quantity', maxIter = 10, regParam=0.3)
lin_reg_model = lin_reg.fit(df_train)

print("Coefficients: " + str(lin_reg_model.coefficients))
print("Intercept: " + str(lin_reg_model.intercept))

Coefficients: [0.002176554432299778,-0.36032287277976727]
Intercept: 80.74728569909968

With a trained model in hand, we can use it to make predictions against the test DataFrame.


In [11]:
df_pred = lin_reg_model.transform(df_test)
display(df_pred)

## Convert model to ONNX
In the cells that follow, we convert the model to ONNX and show how an output of how ONNX represents the Spark ML model.


In [13]:
from onnxmltools import convert_sparkml
from onnxmltools.convert.common.data_types import FloatTensorType

initial_types = [ 
    ("features", FloatTensorType([1, lin_reg_model.numFeatures])),
    # (repeat for the required inputs)
]

In [14]:
model_onnx = convert_sparkml(lin_reg_model, 'sparkml GeneralizedLinearRegression', initial_types)
model_onnx

ir_version: 6
producer_name: "OnnxMLTools"
producer_version: "1.5.5"
domain: "onnxconverter-common"
model_version: 0
doc_string: ""
graph {
  node {
    input: "features"
    output: "prediction"
    name: "LinearRegressor"
    op_type: "LinearRegressor"
    attribute {
      name: "coefficients"
      floats: 0.0021765544079244137
      floats: -0.36032286286354065
      type: FLOATS
    }
    attribute {
      name: "intercepts"
      floats: 80.74728393554688
      type: FLOATS
    }
    domain: "ai.onnx.ml"
  }
  name: "sparkml GeneralizedLinearRegression"
  input {
    name: "features"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 2
          }
        }
      }
    }
  }
  output {
    name: "prediction"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 1
    

## Upload the model to Azure Storage
In the cells that follow we save the ONNX model to the storage of Spark driver node temporarily. Then we use the Azure Storage Python SDK to upload the ONNX model to Azure Storage.


In [15]:
with open("model.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

283

In [18]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=<blob_storage>;AccountKey=<blob_storage_account_key>;EndpointSuffix=core.windows.net"

from azure.storage.blob import BlobClient

blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="models", blob_name="onnx/model.onnx")

with open("./model.onnx", "rb") as data:
    blob.upload_blob(data,overwrite=True)

{'etag': '"0x8D7D05B60F1646C"', 'last_modified': datetime.datetime(2020, 3, 25, 1, 25, 18, tzinfo=datetime.timezone.utc), 'content_md5': bytearray(b'\xe8\xf6U\x18\xbbR\xaf\x05|;\xbea\x19&DM'), 'client_request_id': '7cb73618-6e37-11ea-a65f-000d3a11e0c5', 'request_id': '21c50d23-e01e-0024-2a44-0261c6000000', 'version': '2019-07-07', 'date': datetime.datetime(2020, 3, 25, 1, 25, 17, tzinfo=datetime.timezone.utc), 'request_server_encrypted': True, 'encryption_key_sha256': None, 'encryption_scope': None, 'error_code': None}